Deep Learning Final Project
=============

# §3 Making Predictions on the remaining MNIST Style Greyscale Data
------------

In a previous iPython Notebooks (`0_SVHNDownloadExtract.ipynb` and `1_SVHNExploreProcess32x32.ipynb`), we downloaded the MNIST like 32 x 32 image dataset and then processed it in preparation for model training. In `2_SVHN_TrainPredict.ipynb` we trained and evaluated the performance of a Neural Network using the data contained in the extra_32x32 dataset.

Here we test the performance of this model on the datasets contained in test_32x32 and train_32x32.

## 3.1 Import Modules

In [1]:
from __future__ import print_function
import numpy as np
import time
from IPython.display import display, Image
from six.moves import cPickle as pickle
import tensorflow as tf

## 3.2 Reload the data we generated in `1_SVHNExploreProcess32x32.ipynb`
Supposedly, the extra data examples are 'somewhat less difficult' we'll start with those and see how we do

In [2]:
# Stick all the data into train_dataset and train_labels

with open('data/32x32/greyscale/gs_extra1.pickle', 'rb') as f:
    save = pickle.load(f)
    extra_dataset = save['data']
    extra_labels = save['labels']
    del save  # hint to help gc free up memory
    print('extra_32x32 set 1: ', extra_dataset.shape, extra_labels.shape)
print('')

with open('data/32x32/greyscale/gs_train.pickle', 'rb') as f:
    save = pickle.load(f)
    train_32x32_dataset = save['data']
    train_32x32_labels = save['labels']
    del save  # hint to help gc free up memory
    print('train_32x32 set:', train_32x32_dataset.shape, train_32x32_labels.shape)
print('')

with open('data/32x32/greyscale/gs_test.pickle', 'rb') as f:
    save = pickle.load(f)
    test_32x32_dataset = save['data']
    test_32x32_labels = save['labels']
    del save  # hint to help gc free up memory
    print('test_32x32 set:', test_32x32_dataset.shape, test_32x32_labels.shape)

extra_32x32 set 1:  (177043, 32, 32) (177043,)

train_32x32 set: (73257, 32, 32) (73257,)

test_32x32 set: (26032, 32, 32) (26032,)


In [3]:
# Get the data we used to validation and testing when training our model

split_index = 26556
test_dataset,  test_labels  = extra_dataset[           :  split_index,:,:], extra_labels[           :  split_index]
valid_dataset, valid_labels = extra_dataset[split_index:2*split_index,:,:], extra_labels[split_index:2*split_index]
del extra_dataset, extra_labels

print('test set:', test_dataset.shape, test_labels.shape)
print('valid set:', valid_dataset.shape, test_labels.shape)

print('test_32x32 set:', test_32x32_dataset.shape, test_32x32_labels.shape)
print('train_32x32 set:', train_32x32_dataset.shape, train_32x32_labels.shape)

test set: (26556, 32, 32) (26556,)
valid set: (26556, 32, 32) (26556,)
test_32x32 set: (26032, 32, 32) (26032,)
train_32x32 set: (73257, 32, 32) (73257,)


In [4]:
image_size = 32
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

test_dataset, test_labels = reformat(test_dataset, test_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)

test_32x32_dataset, test_32x32_labels = reformat(test_32x32_dataset, test_32x32_labels)
train_32x32_dataset, train_32x32_labels = reformat(train_32x32_dataset, train_32x32_labels)

print('test set:', test_dataset.shape, test_labels.shape)
print('valid set:', valid_dataset.shape, test_labels.shape)

print('test_32x32 set:', test_32x32_dataset.shape, test_32x32_labels.shape)
print('train_32x32 set:', train_32x32_dataset.shape, train_32x32_labels.shape)

test set: (26556, 1024) (26556, 10)
valid set: (26556, 1024) (26556, 10)
test_32x32 set: (26032, 1024) (26032, 10)
train_32x32 set: (73257, 1024) (73257, 10)


## 3.3 Reload the model we saved in `2_SVHNTrainPredict32x32.ipynb` and make predictions

We make predictions on our training and test sets to check we've loaded our model correctly and get the accuracy we found when training. We then make predictions on the additional datasets provided train_32x32 and test_32x32 to see how robust our model is.

In [7]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [8]:
batch_size = 128

with tf.Session() as session:

    new_saver = tf.train.import_meta_graph('3Layer32x32Model.meta')
    new_saver.restore(session, tf.train.latest_checkpoint('./'))

    h1_weights = session.run(tf.get_collection('h1w')[0])
    h1_biases  = session.run(tf.get_collection('h1b')[0])
    h2_weights = session.run(tf.get_collection('h2w')[0])
    h2_biases  = session.run(tf.get_collection('h2b')[0])
    weights    = session.run(tf.get_collection('w'  )[0])
    biases     = session.run(tf.get_collection('b'  )[0])

    def predict(data):
        return tf.nn.softmax(tf.matmul(tf.nn.relu(
                             tf.matmul(tf.nn.relu(
                             tf.matmul(data, h1_weights) + h1_biases),
                                             h2_weights) + h2_biases),
                                                weights) + biases).eval()

    sum_test_accuracy = 0
    sum_valid_accuracy = 0
    num_batches = test_labels.shape[0] // batch_size
    t1 = time.time()
    for i in xrange(num_batches):
        test_prediction = predict(test_dataset[i*batch_size:(i+1)*batch_size, :])
        test_accuracy = accuracy(test_prediction, test_labels[i*batch_size:(i+1)*batch_size, :])
        sum_test_accuracy += test_accuracy
        valid_prediction = predict(valid_dataset[i*batch_size:(i+1)*batch_size, :])
        valid_accuracy = accuracy(valid_prediction, valid_labels[i*batch_size:(i+1)*batch_size, :])
        sum_valid_accuracy += valid_accuracy
        if i%10==0:
            t2 = time.time()
            print('test/valid:', i, 'of', num_batches, 'time = ', t2-t1, 'test acc:', test_accuracy,
                                                                         'valid acc:', valid_accuracy)
            t1 = t2
    print('valid_accuracy = ', sum_valid_accuracy/num_batches)
    print('test_accuracy = ' , sum_test_accuracy/num_batches)

    sum_test_accuracy = 0
    num_batches = test_32x32_labels.shape[0] // batch_size
    t1 = time.time()
    for i in xrange(num_batches):
        test_prediction = predict(test_32x32_dataset[i*batch_size:(i+1)*batch_size, :])
        test_accuracy = accuracy(test_prediction, test_32x32_labels[i*batch_size:(i+1)*batch_size, :])
        sum_test_accuracy += test_accuracy
        if i%10==0:
            t2 = time.time()
            print('test_32x32:', i, 'of', num_batches, 'time = ', t2-t1, 'test acc:', test_accuracy)
            t1 = t2
    print('test_32x32_accuracy = ' , sum_test_accuracy/num_batches)

    sum_train_accuracy = 0
    num_batches = train_32x32_labels.shape[0] // batch_size
    for i in xrange(num_batches):
        train_prediction = predict(train_32x32_dataset[i*batch_size:(i+1)*batch_size, :])
        train_accuracy = accuracy(train_prediction, train_32x32_labels[i*batch_size:(i+1)*batch_size, :])
        sum_train_accuracy += train_accuracy
        if i%10==0:
            t2 = time.time()
            print('train_32x32:', i, 'of', num_batches, 'time = ', t2-t1, 'train acc:', train_accuracy)
            t1 = t2
    print('train_32x32_accuracy = ' , sum_train_accuracy/num_batches)

test/valid: 0 of 207 time =  1.46046996117 test acc: 92.1875 valid acc: 86.71875
test/valid: 10 of 207 time =  23.5634129047 test acc: 88.28125 valid acc: 92.96875
test/valid: 20 of 207 time =  31.1093809605 test acc: 89.0625 valid acc: 90.625
test/valid: 30 of 207 time =  45.319892168 test acc: 93.75 valid acc: 93.75
test/valid: 40 of 207 time =  54.4379329681 test acc: 93.75 valid acc: 92.96875
test/valid: 50 of 207 time =  60.6018929482 test acc: 89.0625 valid acc: 95.3125
test/valid: 60 of 207 time =  70.2755680084 test acc: 91.40625 valid acc: 94.53125
test/valid: 70 of 207 time =  87.3022480011 test acc: 96.875 valid acc: 91.40625
test/valid: 80 of 207 time =  102.139631987 test acc: 90.625 valid acc: 95.3125
test/valid: 90 of 207 time =  110.696629047 test acc: 89.84375 valid acc: 92.96875
test/valid: 100 of 207 time =  130.845287085 test acc: 91.40625 valid acc: 94.53125
test/valid: 110 of 207 time =  127.133752823 test acc: 96.09375 valid acc: 92.96875
test/valid: 120 of 207 t

KeyboardInterrupt: 